# pySpark 설치와 회귀(Regression) 분석해보기
본 챕터의 실습에서는 pySpark 3.2 버전을 Colab 내에서 설치하고, 부동산 데이터 셋를 활용하여 부동산 가격에 영향을 미치는 특징을 취합하여 부동산 가격을 예측한다. 또한, 생성된 결과에 대해 통계분석 및 시각화에 대한 간단한 예제를 사용해본다.

## pySpark import 하기

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
# pyspark 모듈 및 sql 라이브러리 설치
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## 스파크 설치 여부 확인

In [ ]:
spark

## Spark Web UI로 확인

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

##  데이터셋 올리기
BostonHousing.csv 미국 보스톤 부동산 가격정보

In [ ]:
!mkdir data
!wget -q https://github.com/chunsejin/KGProjects_DAU/raw/master/data/BostonHousing.csv
!mv BostonHousing.csv data


## 회귀분석을 위한 라이브러리 및 데이터셋 로드

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

## 스키마 출력

In [ ]:
dataset.printSchema()

In [ ]:
dataset.show(10)

## 회귀모델 작성
x: 부동산 가격 결정에 영향을 미치는 요인
y: 부동산 가격

In [ ]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

# Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show()

## 회귀모델을 위한 fit 및 예측값 출력



In [ ]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])

regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

회귀분석의 결정계수(intercept)와 상관계수(coefficient)

In [ ]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)


## 기본적인 통계 분석 및 시각화

In [ ]:
x = pred.predictions.select("medv").rdd.flatMap(lambda x: x).collect()
y = pred.predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 

x = np.array(x)
y = np.array(y)

#create basic scatterplot
plt.plot(x, y, 'o')

#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(x, y, 1)

#add linear regression line to scatterplot 
plt.plot(x, m*x+b)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

